In [1]:
# import important packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sys
sys.path.insert(0,'../scripts/')
import warnings
warnings.filterwarnings(action="ignore")

In [2]:
df = pd.read_csv("../data/AdSmartABdata.csv")
df_control= pd.read_csv("../data/control_group_data.csv")
df_exposed= pd.read_csv("../data/exposed_clean_data.csv")

In [4]:
df_control['response'].head()

0    1
1    1
2    1
3    1
4    1
Name: response, dtype: int64

In [5]:
# Dropping observations where there was no response to the presented questionaire
dp = df.query('yes==0 and no==0')
df = df.drop(dp.index)

In [ ]:
# Counting the people that are aware of the brand in both groups